In [2]:
import sys
from math import sqrt
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn import ensemble
from sklearn import linear_model as lm
from sklearn.metrics import mean_squared_error as mse
import xgboost as xgb
import datetime

In [3]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [10]:
data_confirmed = pd.read_csv('data/time_series-ncov-Confirmed.csv')
data_deaths = pd.read_csv('data/time_series-ncov-Deaths.csv')
data_recovered = pd.read_csv('data/time_series-ncov-Recovered.csv')

In [11]:
data_confirmed.head()

,Province/State,Country/Region,Lat,Long,Date,Value
0,#adm1+name,#country+name,#geo+lat,#geo+lon,#date,#affected+infected+value+num
1,NaN,Afghanistan,33,65,2020-03-20,24
2,NaN,Afghanistan,33,65,2020-03-19,22
3,NaN,Afghanistan,33,65,2020-03-18,22
4,NaN,Afghanistan,33,65,2020-03-17,22


In [12]:
data_deaths.head()

,Province/State,Country/Region,Lat,Long,Date,Value
0,#adm1+name,#country+name,#geo+lat,#geo+lon,#date,#affected+killed+value+num
1,NaN,Afghanistan,33,65,2020-03-20,0
2,NaN,Afghanistan,33,65,2020-03-19,0
3,NaN,Afghanistan,33,65,2020-03-18,0
4,NaN,Afghanistan,33,65,2020-03-17,0


In [13]:
data_recovered.head()

,Province/State,Country/Region,Lat,Long,Date,Value
0,#adm1+name,#country+name,#geo+lat,#geo+lon,#date,#affected+recovered+value+num
1,NaN,Afghanistan,33,65,2020-03-20,1
2,NaN,Afghanistan,33,65,2020-03-19,1
3,NaN,Afghanistan,33,65,2020-03-18,1
4,NaN,Afghanistan,33,65,2020-03-17,1


In [14]:
data_confirmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28144 entries, 0 to 28143
Data columns (total 6 columns):
Province/State    19117 non-null object
Country/Region    28144 non-null object
Lat               28144 non-null object
Long              28144 non-null object
Date              28144 non-null object
Value             28144 non-null object
dtypes: object(6)
memory usage: 1.3+ MB


In [19]:
data_confirmed[pd.isnull(data_confirmed['Province/State'])]

,Province/State,Country/Region,Lat,Long,Date,Value
1,NaN,Afghanistan,33,65,2020-03-20,24
2,NaN,Afghanistan,33,65,2020-03-19,22
3,NaN,Afghanistan,33,65,2020-03-18,22
4,NaN,Afghanistan,33,65,2020-03-17,22
5,NaN,Afghanistan,33,65,2020-03-16,21
6,NaN,Afghanistan,33,65,2020-03-15,16
7,NaN,Afghanistan,33,65,2020-03-14,11
8,NaN,Afghanistan,33,65,2020-03-13,7
9,NaN,Afghanistan,33,65,2020-03-12,7
10,NaN,Afghanistan,33,65,2020-03-11,7


In [ ]:
## Grouping Data on Country level (neglecting )